In [1]:
import numpy as np
from cellpose import models, core, io
from spotiflow.model import Spotiflow
from pathlib import Path
from skimage.measure import regionprops_table
import os
import apoc
from tqdm import tqdm
import pyclesperanto_prototype as cle 
from tifffile import imwrite, imread
import pandas as pd
from utils import list_images, read_image, filter_points_interpolated, count_points_in_labels

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Activate .cle GPU acceleration
cle.select_device("gpu")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)
spotiflow_model = Spotiflow.from_pretrained("general")



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




c:\Users\adiez_cmic\miniforge3\envs\hcs_cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


creating new log file
2025-09-29 13:33:17,848 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2025-09-29 13:33:17,849 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0
2025-09-29 13:33:17,905 [INFO] ** TORCH CUDA version installed and working. **
2025-09-29 13:33:17,907 [INFO] ** TORCH CUDA version installed and working. **
2025-09-29 13:33:17,908 [INFO] >>>> using GPU (CUDA)


c:\Users\adiez_cmic\miniforge3\envs\hcs_cellpose\lib\site-packages\pyclesperanto_prototype\_tier0\_device.py:77: UserWarning: No OpenCL device found with gpu in their name. Using NVIDIA GeForce RTX 4090 instead.
  warnings.warn(f"No OpenCL device found with {name} in their name. Using {device.name} instead.")


2025-09-29 13:33:18,817 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cpsam
INFO:spotiflow.model.spotiflow:Loading pretrained model: general
2025-09-29 13:33:19,195 [INFO] Loading pretrained model: general


In [2]:
# Load pretrained Object and Pixel Classifiers
mtb_cl_filename = "./raw_data/SD_DAPI_Mtb_detection_training/Mtb_segmenter.cl"
mtb_segmenter = apoc.ObjectSegmenter(opencl_filename=mtb_cl_filename)

gal3_cl_filename = "./raw_data/SD_RFP_spot_detection_training/Gal3_segmenter.cl"
gal3_segmenter = apoc.PixelClassifier(opencl_filename=gal3_cl_filename)

# Create a list of subdirectories containing the Nuc string (stained for nuclei)
main_directory_path = Path("X:\Lisa\siMtb screen I_LØ")

folders_with_nuc = [
    name for name in os.listdir(main_directory_path)
    if os.path.isdir(os.path.join(main_directory_path, name)) and "Nuc" in name
]

print(folders_with_nuc)

['Plate 01_Nuc', 'Plate 02_Nuc', 'Plate 03_Nuc', 'Plate 04_Nuc', 'Plate 05_Nuc', 'Plate 06_Nuc', 'Plate 07_Nuc', 'Plate 08_Nuc', 'Plate 09_Nuc', 'Plate 10_Nuc', 'Plate 11_Nuc', 'Plate 12_Nuc', 'Plate 13_Nuc', 'Plate 17_Nuc', 'Plate 14_Nuc', 'Plate 15_Nuc', 'Plate 16_Nuc']


In [3]:
# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr),(..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("ki67", 0), ("neun", 1)]

markers = [("LC3B", 0), ("GAL3", 1), ("Chmp4B", 2), ("Mtb", 3)]

In [ ]:
for folder in tqdm(folders_with_nuc):

    print(f"Analyzing Plate: {folder}")

    # Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
    directory_path = Path(f"X:\Lisa\siMtb screen I_LØ\{folder}")

    # Iterate through the .czi and .nd2 files in the directory
    images = list_images(directory_path)

    # Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
    slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

    #TODO: Substract uneven and remove background from BF by obtaining the median of all BF channels 

    try:

        bf_correction = imread(directory_path / "bf_correction.tiff")

    except FileNotFoundError:

        # Create an empty list to store the brightfield images from each well
        bf_arrays = []

        # Read all images, extract the brightfield channel and calculate the mean to correct illumination and remove dust spots
        for image in tqdm(images):

            # Read image, apply slicing if needed and return filename and img as a np array
            img, filename = read_image(image, slicing_factor_xy, log=False)

            # Extract brighfield slice
            bf_channel = img[4]

            # Add it to the bf_arrays iterable
            bf_arrays.append(bf_channel)

        # Create a stack containing all bf images
        bf_stack = np.stack(bf_arrays, axis=0)

        # Calculate the median to retain the common structures (spots, illumination)
        bf_correction = np.median(bf_stack, axis=0)
        del bf_stack

        # Store brightfield correction as .tiff to avoid recalculating it everytime
        imwrite(directory_path / "bf_correction.tiff",bf_correction)

    # Empty list to populate with per well features
    per_well_props = []

    # Empty list to populate with per infection stats
    infection_stats = []

    for image in tqdm(images):

        # Read image, apply slicing if needed and return filename and img as a np array
        img, filename = read_image(image, slicing_factor_xy)

        # Extract plate number and well_id
        plate_nr = filename.split("_")[0]
        well_id = filename.split("Wells-")[1].split("__")[0]

        cytoplasm_labels, flows, styles = model.eval(np.stack((img[[0,1]].sum(axis=0), (img[4] - bf_correction)), axis=0), niter=1000) # need to check the arguments

        mtb_labels = mtb_segmenter.predict(img[3])
        mtb_labels = cle.pull(mtb_labels)

        # Convert mtb_labels to boolean mask
        mtb_boolean = mtb_labels.astype(bool)

        # Use NumPy's indexing to identify labels that intersect with mtb_boolean (bacterial mask)
        infected_labels = np.unique(cytoplasm_labels[mtb_boolean])
        infected_labels = infected_labels[infected_labels != 0]

        infected_mask = np.isin(cytoplasm_labels, infected_labels)
        non_infected_mask = np.isin(cytoplasm_labels, infected_labels, invert=True)
        infected_cytoplasm = np.where(infected_mask, cytoplasm_labels, 0).astype(cytoplasm_labels.dtype)
        non_infected_cytoplasm = np.where(non_infected_mask, cytoplasm_labels, 0).astype(cytoplasm_labels.dtype)

        infected_cells = len(np.unique(infected_cytoplasm)) - (0 in infected_cytoplasm)
        non_infected_cells = len(np.unique(non_infected_cytoplasm)) - (0 in non_infected_cytoplasm)
        total_cells = cytoplasm_labels.max()

        # Calculate percentage of infected cells
        perc_inf_cells = round(infected_cells / total_cells * 100, 2) if total_cells > 0 else 0

        print(f"Non-infected: {non_infected_cells}")
        print(f"Infected: {infected_cells}")
        print(f"Total cells: {total_cells}")
        print(f"Percentage infected:{perc_inf_cells}")

        # Create a dictionary containing all extracted info per image
        stats_dict = {
                    "plate": plate_nr,
                    "well_id": well_id,
                    "total_nr_cells": total_cells,
                    "infected": infected_cells,
                    "non-infected": non_infected_cells,
                    "%_inf_cells": perc_inf_cells 
        }

        # Append the current data point to the stats_list
        infection_stats.append(stats_dict)

        # ----- punctae filtering and counting ------ (for now just GAL3), update later for all the other markers

        # Obtaining GAL3 points 
        points, details = spotiflow_model.predict(img[1], subpix=True)

        # Defining GAL3 signal mask
        gal3_mask = gal3_segmenter.predict(img[1])

        # Filter the predicted Spotiflow points near GAL3 mask
        filtered_points = filter_points_interpolated(points, gal3_mask)

        # Count how many points per cell
        punctae_counts_df = count_points_in_labels(filtered_points, cytoplasm_labels)

        # Append marker name to num_points column name
        punctae_counts_df.rename(columns={"num_points": "GAL3_num_points"}, inplace=True)

        # ----- per_label intensity information ------

        # Empty list to populate with per channel intensity information
        props_list = []

        # Create a dictionary containing all iamge metadata
        descriptor_dict = {
            "plate": plate_nr,
            "well_id": well_id,
            "filepath": image
            }
        for channel_name, ch_nr in tqdm(markers):

            # Extract intensity information from each channel
            props = regionprops_table(label_image=cytoplasm_labels,
                            intensity_image=img[ch_nr],
                            properties=["label", "intensity_mean", "intensity_max", "intensity_min", "intensity_std"])
            
            # Convert to dataframe
            props_df = pd.DataFrame(props)
            
            # Rename intensity columns to human readable format
            props_df.rename(columns={"intensity_mean": f"Intensity_MeanIntensity_Cytoplasm_{channel_name}_ch{ch_nr}"}, inplace=True)
            props_df.rename(columns={"intensity_max": f"Intensity_MaxIntensity_Cytoplasm_{channel_name}_ch{ch_nr}"}, inplace=True)
            props_df.rename(columns={"intensity_min": f"Intensity_MinIntensity_Cytoplasm_{channel_name}_ch{ch_nr}"}, inplace=True)
            props_df.rename(columns={"intensity_std": f"Intensity_StdIntensity_Cytoplasm_{channel_name}_ch{ch_nr}"}, inplace=True)

            # Append each props_df to props_list
            props_list.append(props_df)

        # Initialize the df with the first df in the list
        props_df = props_list[0]
        # Start looping from the second df in the list
        for df in props_list[1:]:
            props_df = props_df.merge(df, on="label")

        # Add each key-value pair from descriptor_dict to props_df at the specified position
        insertion_position = 0
        for key, value in descriptor_dict.items():
            props_df.insert(insertion_position, key, value)
            insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

        #TODO: Merge punctae counts with the props_df containing intensity information
        props_df = props_df.merge(
            punctae_counts_df,
            on="label",
            how="left"   # keep all labels from props_df
        )

        # Fill missing counts with zero
        props_df["GAL3_num_points"] = props_df["GAL3_num_points"].fillna(0).astype(int)

        # Rename label(id) to CellProfiler format ObjectNumber
        props_df.rename(columns={"label": "ObjectNumber"}, inplace=True)

        # Add infected flag to props_df if labels is in infected_labels
        # Find position of "ObjectNumber" column
        col_idx = props_df.columns.get_loc("ObjectNumber")

        # Insert new column right after "ObjectNumber"
        props_df.insert(col_idx + 1, "Mtb_infected", props_df["ObjectNumber"].isin(infected_labels))

        # Append each props_df to per_well_props
        per_well_props.append(props_df)

    # Transform into a dataframe to store it as .csv
    df = pd.DataFrame(infection_stats)
    df.to_csv(f"./results_infection_{plate_nr}.csv")

    # Concatenate all per_well_props into final_df
    final_df = pd.concat(per_well_props, ignore_index=True)
    final_df.to_csv(f"./results_per_label_{plate_nr}.csv")

In [4]:
folder = folders_with_nuc[0]

In [19]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(f"X:\Lisa\siMtb screen I_LØ\{folder}")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

bf_correction = imread(directory_path / "bf_correction.tiff")

In [22]:
image = images[0]

# Empty list to populate with per well features
per_well_props = []

# Empty list to populate with per infection stats
infection_stats = []

In [23]:
image

'X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A1__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2'

In [18]:
import napari
viewer = napari.Viewer(ndisplay=2)

In [ ]:
# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract plate number and well_id
plate_nr = filename.split("_")[0]
well_id = filename.split("Wells-")[1].split("__")[0]

cytoplasm_labels, flows, styles = model.eval(np.stack((img[[0,1]].sum(axis=0), (img[4] - bf_correction)), axis=0), niter=1000) # need to check the arguments

mtb_labels = mtb_segmenter.predict(img[3])
mtb_labels = cle.pull(mtb_labels)

# Convert mtb_labels to boolean mask
mtb_boolean = mtb_labels.astype(bool)

# Use NumPy's indexing to identify labels that intersect with mtb_boolean (bacterial mask)
infected_labels = np.unique(cytoplasm_labels[mtb_boolean])
infected_labels = infected_labels[infected_labels != 0]

infected_mask = np.isin(cytoplasm_labels, infected_labels)
non_infected_mask = np.isin(cytoplasm_labels, infected_labels, invert=True)
infected_cytoplasm = np.where(infected_mask, cytoplasm_labels, 0).astype(cytoplasm_labels.dtype)
non_infected_cytoplasm = np.where(non_infected_mask, cytoplasm_labels, 0).astype(cytoplasm_labels.dtype)

infected_cells = len(np.unique(infected_cytoplasm)) - (0 in infected_cytoplasm)
non_infected_cells = len(np.unique(non_infected_cytoplasm)) - (0 in non_infected_cytoplasm)
total_cells = cytoplasm_labels.max()

# Calculate percentage of infected cells
perc_inf_cells = round(infected_cells / total_cells * 100, 2) if total_cells > 0 else 0

print(f"Non-infected: {non_infected_cells}")
print(f"Infected: {infected_cells}")
print(f"Total cells: {total_cells}")
print(f"Percentage infected:{perc_inf_cells}")

# Create a dictionary containing all extracted info per image
stats_dict = {
            "plate": plate_nr,
            "well_id": well_id,
            "total_nr_cells": total_cells,
            "infected": infected_cells,
            "non-infected": non_infected_cells,
            "%_inf_cells": perc_inf_cells 
}

# Append the current data point to the stats_list
infection_stats.append(stats_dict)

# ----- punctae filtering and counting ------ (for now just GAL3), update later for all the other markers

# Obtaining GAL3 points 
points, details = spotiflow_model.predict(img[1], subpix=True)
viewer.add_image(img[1])
viewer.add_points(points, face_color='red')

# Defining GAL3 signal mask
gal3_mask = gal3_segmenter.predict(img[1])
viewer.add_labels(gal3_mask)
viewer.add_labels(cytoplasm_labels)

# Filter the predicted Spotiflow points near GAL3 mask
filtered_points = filter_points_interpolated(points, gal3_mask)
viewer.add_points(filtered_points, face_color='blue')

# Count how many points per cell
punctae_counts_df = count_points_in_labels(filtered_points, cytoplasm_labels)

# Append marker name to num_points column name
punctae_counts_df.rename(columns={"num_points": "GAL3_num_points"}, inplace=True)

# ----- per_label intensity information ------

# Empty list to populate with per channel intensity information
props_list = []

# Create a dictionary containing all iamge metadata
descriptor_dict = {
    "plate": plate_nr,
    "well_id": well_id,
    "filepath": image
    }
for channel_name, ch_nr in tqdm(markers):

    # Extract intensity information from each channel
    props = regionprops_table(label_image=cytoplasm_labels,
                    intensity_image=img[ch_nr],
                    properties=["label", "intensity_mean", "intensity_max", "intensity_min", "intensity_std"])
    
    # Convert to dataframe
    props_df = pd.DataFrame(props)
    
    # Rename intensity columns to human readable format
    props_df.rename(columns={"intensity_mean": f"Intensity_MeanIntensity_Cytoplasm_{channel_name}_ch{ch_nr}"}, inplace=True)
    props_df.rename(columns={"intensity_max": f"Intensity_MaxIntensity_Cytoplasm_{channel_name}_ch{ch_nr}"}, inplace=True)
    props_df.rename(columns={"intensity_min": f"Intensity_MinIntensity_Cytoplasm_{channel_name}_ch{ch_nr}"}, inplace=True)
    props_df.rename(columns={"intensity_std": f"Intensity_StdIntensity_Cytoplasm_{channel_name}_ch{ch_nr}"}, inplace=True)

    # Append each props_df to props_list
    props_list.append(props_df)

# Initialize the df with the first df in the list
props_df = props_list[0]
# Start looping from the second df in the list
for df in props_list[1:]:
    props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

#TODO: Merge punctae counts with the props_df containing intensity information
props_df = props_df.merge(
    punctae_counts_df,
    on="label",
    how="left"   # keep all labels from props_df
)

# Fill missing counts with zero
props_df["GAL3_num_points"] = props_df["GAL3_num_points"].fillna(0).astype(int)

# Rename label(id) to CellProfiler format ObjectNumber
props_df.rename(columns={"label": "ObjectNumber"}, inplace=True)

# Add infected flag to props_df if labels is in infected_labels
# Find position of "ObjectNumber" column
col_idx = props_df.columns.get_loc("ObjectNumber")

# Insert new column right after "ObjectNumber"
props_df.insert(col_idx + 1, "Mtb_infected", props_df["ObjectNumber"].isin(infected_labels))

# Append each props_df to per_well_props
per_well_props.append(props_df)

# Transform into a dataframe to store it as .csv
df = pd.DataFrame(infection_stats)
df.to_csv(f"./results_infection_{plate_nr}_test.csv")

# Concatenate all per_well_props into final_df
final_df = pd.concat(per_well_props, ignore_index=True)
final_df.to_csv(f"./results_per_label_{plate_nr}_test.csv")



Image analyzed: Plate01_Nuc_Wells-A1__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR
Original Array shape: (6, 5032, 5032)
Compressed Array shape: (6, 5032, 5032)
Non-infected: 2221
Infected: 599
Total cells: 2820
Percentage infected:21.24
INFO:spotiflow.model.spotiflow:Will use device: cuda:0
2025-09-29 13:57:16,815 [INFO] Will use device: cuda:0
INFO:spotiflow.model.spotiflow:Predicting with prob_thresh = [0.6], min_distance = 1
2025-09-29 13:57:16,815 [INFO] Predicting with prob_thresh = [0.6], min_distance = 1
INFO:spotiflow.model.spotiflow:Peak detection mode: fast
2025-09-29 13:57:16,815 [INFO] Peak detection mode: fast
INFO:spotiflow.model.spotiflow:Image shape (5032, 5032)
2025-09-29 13:57:16,815 [INFO] Image shape (5032, 5032)
INFO:spotiflow.model.spotiflow:Predicting with (3, 3) tiles
2025-09-29 13:57:16,815 [INFO] Predicting with (3, 3) tiles
INFO:spotiflow.model.spotiflow:Normalizing...
2025-09-29 13:57:16,845 [INFO] Normalizing...
INFO:spotiflow.model.spotiflow:Padd

Predicting tiles: 100%|██████████| 9/9 [00:02<00:00,  3.19it/s]

INFO:spotiflow.model.spotiflow:Found 822 spots
2025-09-29 13:57:20,016 [INFO] Found 822 spots



100%|██████████| 4/4 [00:00<00:00,  5.80it/s]
